### Install PYTHON dependencies

In [1]:
pip install fhirpy Authlib python-dotenv

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
import datetime
import os
import uuid

import requests
from authlib.jose import jwt
from dotenv import load_dotenv

from fhirpy import SyncFHIRClient

### load environment variables

`cp example/.env.tpl example/.env`   
`CLIENT_ID` - your epic app Non-Production Client ID   
`PRIVATE_KEY_PATH` - path to your private .pem file (see [documentation](https://fhir.epic.com/Documentation?docId=oauth2&section=Creating-Key-Pair))

In [3]:
load_dotenv("./example/.env")

True

### Generate JWT and get access token

In [62]:
BASE_URL = "https://fhir.epic.com/interconnect-fhir-oauth"

In [63]:
jwt_header = {"alg": "RS384", "typ": "JWT"}

In [64]:
# Token expiration time should not be more then 5 minutes since the current
exp = int((datetime.datetime.now() + datetime.timedelta(minutes=4)).timestamp())

payload = {
    "iss": os.getenv("CLIENT_ID"),
    "sub": os.getenv("CLIENT_ID"),
    "aud": f"{BASE_URL}/oauth2/token",
    "jti": str(uuid.uuid4()),
    "exp": exp,
}

In [65]:
with open(os.getenv("PRIVATE_KEY_PATH")) as f:
    private_key = f.read()

In [66]:
encoded_token = jwt.encode(jwt_header, payload, private_key)

In [67]:
headers = requests.structures.CaseInsensitiveDict(
    [("Content-Type", "application/x-www-form-urlencoded")]
)

In [68]:
data = {
    "grant_type": "client_credentials",
    "client_assertion_type": "urn:ietf:params:oauth:client-assertion-type:jwt-bearer",
    "client_assertion": encoded_token
}

In [69]:
response = requests.post(
    f"{BASE_URL}/oauth2/token",
    headers=headers,
    data=data,
)

In [70]:
access_token = response.json()["access_token"]

### Init FHIR Client

In [72]:
FHIR_BASE_URL = f"{BASE_URL}/api/FHIR/R4"

In [73]:
client = SyncFHIRClient(
    FHIR_BASE_URL,
    authorization=f"Bearer {access_token}",
)

### Fetch resources

In [74]:
client.resources("Patient").search(family="Mychart").fetch()

[]

In [75]:
client.execute("Patient/eJzlzKe3KPzAV5TtkxmNivQ3", method="GET")

{'resourceType': 'Patient',
 'id': 'eJzlzKe3KPzAV5TtkxmNivQ3',
 'extension': [{'extension': [{'valueCoding': {'system': 'http://terminology.hl7.org/CodeSystem/v3-NullFlavor',
      'code': 'UNK',
      'display': 'Unknown'},
     'url': 'ombCategory'},
    {'valueString': 'Unknown', 'url': 'text'}],
   'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-race'},
  {'extension': [{'valueString': 'Unknown', 'url': 'text'}],
   'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity'},
  {'valueCodeableConcept': {'coding': [{'system': 'urn:oid:1.2.840.114350.1.13.0.1.7.10.698084.130.657370.19999000',
      'code': 'female',
      'display': 'female'}]},
   'url': 'http://open.epic.com/FHIR/StructureDefinition/extension/legal-sex'},
  {'valueCodeableConcept': {'coding': [{'system': 'urn:oid:1.2.840.114350.1.13.0.1.7.10.698084.130.657370.19999000',
      'code': 'female',
      'display': 'female'}]},
   'url': 'http://open.epic.com/FHIR/StructureDefinition/exte

In [76]:
client.execute("Patient?gender=female", method="GET")

{'resourceType': 'Bundle',
 'type': 'searchset',
 'total': 0,
 'link': [{'relation': 'self',
   'url': 'https://fhir.epic.com/interconnect-fhir-oauth/api/FHIR/R4/Patient?gender=female?'}],
 'entry': [{'fullUrl': 'urn:uuid:00000000-0010-5c13-5bfd-cc9613034941',
   'resource': {'resourceType': 'OperationOutcome',
    'issue': [{'severity': 'information',
      'code': 'invalid',
      'details': {'coding': [{'system': 'urn:oid:1.2.840.114350.1.13.0.1.7.2.657369',
         'code': '59100',
         'display': 'Content invalid against the specification or a profile.'}],
       'text': 'Content invalid against the specification or a profile.'},
      'diagnostics': 'Optional gender invalid: female?',
      'location': ['/f:gender'],
      'expression': ['gender']},
     {'severity': 'warning',
      'code': 'processing',
      'details': {'coding': [{'system': 'urn:oid:1.2.840.114350.1.13.0.1.7.2.657369',
         'code': '4101',
         'display': 'Resource request returns no results.'}],

In [77]:
client.resources("Patient").search(address=123).fetch()

[]